In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt 
import cv2 
import imutils
from os import listdir
import os
%matplotlib inline

import sys
# sys.executable
import pandas as pd
import keras as K
from  keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

# set the matplotlib backend so figures can be saved in the background
import matplotlib
# matplotlib.use("Agg")
 
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
# from pyimagesearch.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
import pandas as pd

import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.models import load_model

In [6]:
!unzip '/content/drive/My Drive/Collab  Code/Gala Challenge /dataset.zip';

Streaming output truncated to the last 5000 lines.
  inflating: dataset/train_flipped/image1842_flip.jpg  
  inflating: dataset/train_flipped/image1843_flip.jpg  
  inflating: dataset/train_flipped/image1844_flip.jpg  
  inflating: dataset/train_flipped/image1845_flip.jpg  
  inflating: dataset/train_flipped/image1849_flip.jpg  
  inflating: dataset/train_flipped/image1851_flip.jpg  
  inflating: dataset/train_flipped/image1852_flip.jpg  
  inflating: dataset/train_flipped/image1854_flip.jpg  
  inflating: dataset/train_flipped/image1859_flip.jpg  
  inflating: dataset/train_flipped/image186_flip.jpg  
  inflating: dataset/train_flipped/image1860_flip.jpg  
  inflating: dataset/train_flipped/image1861_flip.jpg  
  inflating: dataset/train_flipped/image1862_flip.jpg  
  inflating: dataset/train_flipped/image1863_flip.jpg  
  inflating: dataset/train_flipped/image1865_flip.jpg  
  inflating: dataset/train_flipped/image1868_flip.jpg  
  inflating: dataset/train_flipped/image187_flip.jpg  

In [0]:
"""Use below line to run notebookk locally """
# train_img_path='../dataset/Train Images/'
# test_img_path='../dataset/Test Images/'
# Train_csv_path='../dataset/train.csv'
# Test_csv_path='../dataset/test.csv'

"""below line to run notebook on collab"""
train_img_path='./dataset/Train Images/'
test_img_path='./dataset/Test Images/'
Train_csv_path='./dataset/original_and_flipped.csv'
Test_csv_path='./dataset/test.csv'
model_path='./content/model10.model'
flipped_img_path='./dataset/train_flipped/'
output_path='./output/'

image_dims=[224,224,3] 
"""suggested [60 80 3]"""
batch_size=32
epochs=10
lr=1e-2
args={}
args['model']=None
args['output']=output_path


In [8]:
data=pd.read_csv(Train_csv_path)
data.head(2)

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc


In [9]:
dummy_cols=pd.Series(data['Class'].unique())
label_dict=pd.get_dummies(dummy_cols).astype(int)
label_dict

,Attire,Decorationandsignage,Food,misc
0,0,0,1,0
1,0,0,0,1
2,1,0,0,0
3,0,1,0,0


In [10]:
data.shape

(11966, 2)

In [0]:
# imgs=[]
from random import shuffle
for i,file in enumerate(listdir(train_img_path)):
#     print(file)
    img=cv2.imread(train_img_path+file)
    print(img.shape)
    # imgs.append(img)
    if i ==40:
        break     

In [0]:
# plt.figure(figsize=(10,25))
# col=5
# for i,img in enumerate(imgs):
#     plt.subplot(len(imgs)/col+1,col,i+1)
#     plt.imshow(img)

In [0]:
class CNN:
    @staticmethod
    def build(width, height, depth, outputshape):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        # CONV => RELU => POOL

        inputShape = (height, width, depth)
        chanDim = -1

        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
        model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3)))
        # model.add(Dropout(0.25))

        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        # model.add(Dropout(0.25))

        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        # model.add(Dropout(0.25))
        		# first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation("relu"))
        model.add(BatchNormalization())

        model.add(Dense(512))
        model.add(Activation("relu"))
        model.add(BatchNormalization())

        model.add(Dense(256))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        # model.add(Dropout(0.5))

        # 
        model.add(Dense(outputshape))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return model

In [0]:
class VGG:
  @staticmethod
  def build(width, height, depth, output_shape):
    custom_vgg = Sequential()
    custom_vgg.add(Conv2D(32, (3, 3), strides = 1, padding = "same", activation = "relu", input_shape = (width, height,depth)))
    custom_vgg.add(Dropout(0.4))
    custom_vgg.add(Conv2D(32, (3, 3), strides = 1, padding = "same", activation = "relu"))
    custom_vgg.add(Dropout(0.4))
    custom_vgg.add(MaxPooling2D((2, 2)))

    custom_vgg.add(Conv2D(64, (3, 3), strides = 1, padding = "same", activation = "relu"))
    custom_vgg.add(Dropout(0.4))
    custom_vgg.add(Conv2D(64, (3, 3), strides = 1, padding = "same", activation = "relu"))
    custom_vgg.add(Dropout(0.4))
    custom_vgg.add(MaxPooling2D((2, 2)))

    custom_vgg.add(Conv2D(128, (3, 3), strides = 1, padding = "same", activation = "relu"))
    custom_vgg.add(Dropout(0.4))
    custom_vgg.add(Conv2D(128, (3, 3), strides = 1, padding = "same", activation = "relu"))
    custom_vgg.add(Dropout(0.4))
    custom_vgg.add(MaxPooling2D((2, 2)))


    custom_vgg.add(Conv2D(256, (3, 3), strides = 1, padding = "same", activation = "relu"))
    custom_vgg.add(Dropout(0.4))
    custom_vgg.add(Conv2D(256, (3, 3), strides = 1, padding = "same", activation = "relu"))
    custom_vgg.add(Dropout(0.4))
    custom_vgg.add(MaxPooling2D((2, 2)))

    custom_vgg.add(Conv2D(512, (3, 3), strides = 1, padding = "same", activation = "relu"))
    custom_vgg.add(Dropout(0.4))
    custom_vgg.add(Conv2D(512, (3, 3), strides = 1, padding = "same", activation = "relu"))
    custom_vgg.add(Dropout(0.4))
    custom_vgg.add(MaxPooling2D((2, 2)))

    custom_vgg.add(Flatten())

    custom_vgg.add(Dense(1024, activation = "relu"))
    custom_vgg.add(Dropout(0.4))
    custom_vgg.add(Dense(1024, activation = "relu"))
    custom_vgg.add(Dropout(0.4))

    custom_vgg.add(Dense(512, activation = "relu"))
    custom_vgg.add(Dropout(0.2))
    custom_vgg.add(Dense(512, activation = "relu"))
    custom_vgg.add(Dropout(0.2))

    custom_vgg.add(Dense(output_shape, activation = "softmax"))

  # custom_vgg.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
    return custom_vgg


In [0]:
# for i in listdir(flipped_img_path):
#   img=cv2.imread(flipped_img_path+i)
#   print(flipped_img_path+i)
#   img.shape
#   break

./dataset/train_flipped/image8219_flip.jpg


In [0]:
from keras.applications.vgg16 import preprocess_input
class generator():
  def __init__(self,mode='train', csv_path=Train_csv_path,flip_img_dir=flipped_img_path, img_dir=train_img_path, label_dict=label_dict):
    self.mode=mode
    self.image_paths=sorted(list(listdir(img_dir)))
    self.image_paths+=sorted(list(listdir(flip_img_dir)))
    self.img_dir=[img_dir,flip_img_dir]
    self.label_dict=label_dict
    self.data=pd.read_csv(csv_path)
    self.index=0
    self.csv_data=dict(zip(self.data['Image'].values,self.data['Class'].values))
    random.seed(10)
    random.shuffle(self.image_paths)
    if mode=='test':
      self.image_paths=self.image_paths[10000:]
    else:
      self.image_paths=self.image_paths[:10000]

  def generate_image(self,batch_size=batch_size):

      
    while True:
      # index*batch_size<len(image_paths)
      batch_x=[]
      batch_y=[]
      # batch_imgs=np.random.choice(image_paths,size=batch_size)
      batch_imgs= self.image_paths[(self.index)*batch_size:(self.index+1)*batch_size]
      for i in batch_imgs:
        try:
          temp_img=cv2.imread(self.img_dir[0]+i)
          temp_img.shape
          # print('in try ', self.img_dir[0]+i)
        except:
          # print('in catch ', self.img_dir[1]+i)
          temp_img=cv2.imread(self.img_dir[1]+i)
        temp_img.shape
        temp_img=cv2.resize(temp_img,(image_dims[0],image_dims[1]))
        # temp_img -= temp_img.mean()
        # temp_img /= np.maximum(temp_img.std(), 1/image_dims[0]**2)
        img = img_to_array(temp_img)
        # img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        batch_x.append(img)
        batch_y.append(self.label_dict[self.csv_data[i]].values)
        # except:
        #   print('not found')
        #   pass
      x=np.array(batch_x, dtype='float')
      y=np.array(batch_y)
      self.index+=1
      if self.mode=='train' and (self.index)==(10000//batch_size):
        self.index=0
      elif self.mode=='test' and (self.index)==1966//batch_size:
        self.index=0
      if x.shape[0]!=32:
        print(x.shape,'  not complete batch ')
      yield (x,y)

In [0]:
from keras.callbacks import Callback
class EpochCheckpoint(Callback):
  def __init__(self,output_path,name,start_at=0,every=5):
    super(Callback,self).__init__()
    self.output_path=output_path
    self.every=every
    self.intEpoch=start_at
    self.name=name
  def on_epoch_end(self,epoch,logs={}):
    if (self.intEpoch+1)%self.every==0:
      self.model.save(self.output_path+f'{self.name}_{self.intEpoch+1}.hdf5',overwrite=True)
    self.intEpoch+=1



In [0]:
def get_custom_model(model_name='vgg',lr=lr):
  if model_name=='vgg':
    model=VGG.build(image_dims[1],image_dims[0],image_dims[2],output_shape=4)
  elif model_name=='cnn':
    model=CNN.build(image_dims[0],image_dims[1],image_dims[2],output_shape=4)    
  opt = Adam(lr=lr)
  model.compile(optimizer=opt, metrics=['accuracy'], loss='categorical_crossentropy')
  return model

In [0]:
def get_model(lr=lr, model=VGG16):
# initialize the model
    print("[INFO] compiling model...")
    base_model=model(include_top=False,weights='imagenet')#,input_shape=(image_dims[1],image_dims[0],image_dims[2]))
    # x=base_model.layers[-1].output
    x=base_model.output
    # x=Flatten()(x)
    x=GlobalAveragePooling2D()(x)
    x=Dense(512,activation='relu')(x) #dense layer 1
    x=Dropout(0.5)(x)
    x=Dense(512,activation='relu')(x) #dense layer 2
    x=Dropout(0.5)(x)
    # x=Dense(512,activation='relu')(x) #dense layer 2
    # x=Dropout(0.5)(x)
    preds=Dense(4,activation='softmax')(x) #dense layer 3
    model=Model(input=base_model.input,outputs=preds)
    for i in model.layers[:-6]:
      i.trainable=False
    opt = Adam(lr=0.00005)#, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer=opt, metrics=['accuracy'], loss='categorical_crossentropy')
    return model


In [0]:
def train(BS=batch_size,EPOCHS=epochs,lr=lr,model_name=ResNet50):# initialize the number of epochs to train for, initial learning rate,
    print("[INFO] training network...")
    if args['model']==None:
      # model=get_model()
      model=get_model(model=model_name)
    else :
      model=load_model(args['model'])
      opt=Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
      for layer in model.layers:
        layer.trainable = True
      model.compile(optimizer=opt, metrics=['accuracy'], loss='categorical_crossentropy')

    print(f'old learning rate:  {K.get_value(model.optimizer.lr)}')
    K.set_value(model.optimizer.lr,lr)
    print(f' training on lr : {K.get_value(model.optimizer.lr)}')  

    """instance for training and testing generator"""

    H = model.fit_generator(generator=train_generator.generate_image(),
        validation_data=test_generator.generate_image(),
        validation_steps=61,
        steps_per_epoch=312,
        callbacks=callbacks,
        epochs=EPOCHS, verbose=1)
    return model,H

In [21]:
args['model']=None
callbacks=[EpochCheckpoint(output_path=args['output'],name='vgg16')]
test_generator=generator(mode='test')
train_generator=generator()
vgg16_10=train(lr=1e-3,EPOCHS=5, model_name=VGG16)

[INFO] training network...
[INFO] compiling model...




58892288/58889256 [==============================] - 1s 0us/step







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


old learning rate:  4.999999873689376e-05
 training on lr : 0.0010000000474974513


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  app.launch_new_instance()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
312/312 [==============================] - 41s 130ms/step - loss: 1.2302 - acc: 0.6293 - val_loss: 0.6450 - val_acc: 0.7649
Epoch 2/5
312/312 [==============================] - 33s 105ms/step - loss: 0.7202 - acc: 0.7234 - val_loss: 0.5699 - val_acc: 0.8007
Epoch 3/5
312/312 [==============================] - 33s 105ms/step - loss: 0.6115 - acc: 0.7699 - val_loss: 0.5175 - val_acc: 0.8186
Epoch 4/5
312/312 [==============================] - 33s 105ms/step - loss: 0.5379 - acc: 0.8000 - val_loss: 0.4784 - val_acc: 0.8340
Epoch 5/5
312/312 [==============================] - 33s 105ms/step - loss: 0.5125 - acc: 0.8105 - val_loss: 0.4581 - val_acc: 0.8320


In [22]:
args['model']='/content/output/vgg16_5.hdf5'
callbacks=[EpochCheckpoint(start_at=5,output_path=args['output'],name='vgg16')]
vgg16_15=train(lr=0.5*1e-4,EPOCHS=10)

[INFO] training network...
old learning rate:  4.999999873689376e-05
 training on lr : 4.999999873689376e-05
Epoch 1/10
312/312 [==============================] - 81s 261ms/step - loss: 0.6754 - acc: 0.7425 - val_loss: 0.5496 - val_acc: 0.8222
Epoch 2/10
312/312 [==============================] - 78s 250ms/step - loss: 0.4864 - acc: 0.8221 - val_loss: 0.5197 - val_acc: 0.7997
Epoch 3/10
312/312 [==============================] - 78s 251ms/step - loss: 0.3883 - acc: 0.8629 - val_loss: 0.4371 - val_acc: 0.8340
Epoch 4/10
312/312 [==============================] - 78s 250ms/step - loss: 0.3010 - acc: 0.8958 - val_loss: 0.4334 - val_acc: 0.8371
Epoch 5/10
312/312 [==============================] - 78s 250ms/step - loss: 0.2575 - acc: 0.9101 - val_loss: 0.4195 - val_acc: 0.8509
Epoch 6/10
312/312 [==============================] - 78s 250ms/step - loss: 0.2064 - acc: 0.9300 - val_loss: 0.3828 - val_acc: 0.8622
Epoch 7/10
312/312 [==============================] - 78s 251ms/step - loss: 0.19

In [23]:
args['model']='/content/output/vgg16_15.hdf5'
callbacks=[EpochCheckpoint(start_at=15,output_path=args['output'],name='vgg16')]
vgg16_25=train(lr=.5*1e-4,EPOCHS=10)

[INFO] training network...
old learning rate:  4.999999873689376e-05
 training on lr : 4.999999873689376e-05
Epoch 1/10
312/312 [==============================] - 81s 259ms/step - loss: 0.1324 - acc: 0.9604 - val_loss: 0.4785 - val_acc: 0.8304
Epoch 2/10
312/312 [==============================] - 79s 252ms/step - loss: 0.1041 - acc: 0.9672 - val_loss: 0.4116 - val_acc: 0.8673
Epoch 3/10
312/312 [==============================] - 78s 251ms/step - loss: 0.1020 - acc: 0.9678 - val_loss: 0.3713 - val_acc: 0.8776
Epoch 4/10
312/312 [==============================] - 79s 252ms/step - loss: 0.0632 - acc: 0.9807 - val_loss: 0.4198 - val_acc: 0.8786
Epoch 5/10
312/312 [==============================] - 79s 252ms/step - loss: 0.0870 - acc: 0.9723 - val_loss: 0.4060 - val_acc: 0.8689
Epoch 6/10
312/312 [==============================] - 79s 252ms/step - loss: 0.0913 - acc: 0.9703 - val_loss: 0.5956 - val_acc: 0.8432
Epoch 7/10
312/312 [==============================] - 79s 252ms/step - loss: 0.07

In [0]:
for i in temp_model.layers:
  print(i.name,'     ', i.trainable)
# temp_model.output.shape

input_41       False
block1_conv1       False
block1_conv2       False
block1_pool       False
block2_conv1       False
block2_conv2       False
block2_pool       False
block3_conv1       False
block3_conv2       False
block3_conv3       False
block3_pool       False
block4_conv1       False
block4_conv2       False
block4_conv3       False
block4_pool       False
block5_conv1       False
block5_conv2       False
block5_conv3       False
block5_pool       True
flatten_15       True
dense_62       True
dropout_30       True
dense_63       True
dropout_31       True
dense_64       True


In [0]:
temp_model=get_model(model=ResNet50)
temp_model.summary()

[INFO] compiling model...
Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_12[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
_________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  app.launch_new_instance()


In [0]:
args['model']='/content/output/vgg19_30.hdf5'
callbacks=[EpochCheckpoint(start_at=30,output_path=args['output'],name='vgg19'),]
vgg16_40=train(lr=1e-5,EPOCHS=10)

[INFO] training network...
old learning rate:  9.999999747378752e-05
 training on lr : 9.999999747378752e-06
Epoch 1/10
318/318 [==============================] - 11s 33ms/step - loss: 0.8377 - acc: 0.6558 - val_loss: 0.9669 - val_acc: 0.5988
Epoch 2/10
318/318 [==============================] - 7s 22ms/step - loss: 0.8397 - acc: 0.6599 - val_loss: 0.9644 - val_acc: 0.5938
Epoch 3/10
318/318 [==============================] - 7s 21ms/step - loss: 0.8420 - acc: 0.6532 - val_loss: 0.9674 - val_acc: 0.5962
Epoch 4/10
318/318 [==============================] - 7s 22ms/step - loss: 0.8276 - acc: 0.6657 - val_loss: 0.9641 - val_acc: 0.5975
Epoch 5/10
318/318 [==============================] - 7s 22ms/step - loss: 0.8298 - acc: 0.6599 - val_loss: 0.9660 - val_acc: 0.5925
Epoch 6/10
318/318 [==============================] - 7s 22ms/step - loss: 0.8331 - acc: 0.6580 - val_loss: 0.9673 - val_acc: 0.5975
Epoch 7/10
318/318 [==============================] - 7s 21ms/step - loss: 0.8312 - acc: 0.6

In [0]:
from keras.models import load_model
def prediction(img_path=test_img_path, model_path=args['model']):
  model=load_model(model_path)
  output=[]
  for img in listdir(img_path):
    temp_img=cv2.imread(img_path+img)
    # print(im.shape)
    temp_img=cv2.resize(temp_img,(image_dims[0],image_dims[1]))
    im = img_to_array(temp_img)
    im = np.expand_dims(im, axis=0)
    im = preprocess_input(im)
    output.append([img,dummy_cols[np.argmax(model.predict(im))]])
  return output
    

In [0]:
args['model']='/content/output/vgg16_15.hdf5'
output_label=prediction(model_path=args['model'])
# # # load_model()
pd.DataFrame(output_label, columns=['Image','Class']).to_csv('sub13.csv',index=False)
